In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def predict(text):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    model.to(device)
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    # predicted = torch.argmax(predicted, dim=1).numpy()
    predicted = torch.argmax(predicted.cpu(), dim=1).numpy()
    return predicted


tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [ ]:
#pprint(predict('🇵🇰🇦🇫 Исламабад сделал ставку на афганских талибов, но она не сыграла. У «Талибана» с Пакистаном по-прежнему остаются серьезные противоречия из-за так называемой линии Дюранда. \n\n- Что это за линия и почему Исламабад и Кабул не могут считаться полноценными союзниками?\n\n- О нынешнем конфликте, его формальной причине и его истоках;\n\nЧитайте в очередном материале для РИА ФАН: https://riafan.ru/23807534-pakistanu_dorogo_obhodyatsya_ego_zaigrivaniya_s_talibanom'))

In [ ]:
print(torch.cuda.is_available())


True


In [ ]:
# !pip install polyglot
# !pip install pyicu
# !pip install pycld2
# !pip install morfessor

In [ ]:
# from polyglot.downloader import downloader
# from polyglot.text import Text
# from tqdm.auto import tqdm

In [ ]:
# downloader.supported_tasks()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import logging
DATA_FOLDER = "/content/drive/MyDrive/Colab_files/HW4-6_python_basics/nlp_v1"
INPUT_FOLDER = f"{DATA_FOLDER}/input_data"
OUTPUT_FOLDER = f"{DATA_FOLDER}/output_data"
logging.basicConfig(filename="nlp_v1.log")

In [ ]:
# make a list of all the channel files
channels = []
for filename in tqdm(os.listdir(INPUT_FOLDER), desc='channels', unit='filenames'):
    if filename.endswith('.csv'):
        channels.append(filename)

channels: 100%|██████████| 299/299 [00:00<00:00, 255677.25filenames/s]


In [ ]:
def read_channel(filename):
    dtype_dict = {'id': 'int64', 'views': 'float64'}
    df = pd.read_csv(filename, index_col='Unnamed: 0', dtype=dtype_dict, low_memory=False)
    return df

In [ ]:
# test_df = read_channel(f"{INPUT_FOLDER}/{channels[0]}")
# test_df.head()

In [ ]:
def preprocess_df(df, name):
    df['name'] = name[:-4]
    try:
      df.drop(['msg_entity'], axis=1, inplace=True)
    except:
      pass

In [ ]:
# preprocess_df(test_df, channels[0])
# test_df.head()

In [ ]:
def exc_handler(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            #logging.ERROR(f"NaN handler interception: {e}")
            print(f"NaN handler interception: {e}")
            return np.nan
    return wrapper

In [ ]:
# small_df = test_df.head(100).copy(deep=True)

In [ ]:
# tqdm.pandas()
# small_df['sentiment'] = small_df['message'].progress_apply(lambda x: predict(x) if pd.notna(x) else np.nan)

In [ ]:
# small_df.head(10)

0: NEUTRAL
1: POSITIVE
2: NEGATIVE

In [ ]:
# tqdm.pandas()
# test_df['sentiment'] = test_df['message'].progress_apply(lambda x: exc_handler(predict)(x) if pd.notna(x) else np.nan)

In [ ]:
#test_df.iloc[10:15]

In [ ]:
def process_df(df):
  tqdm.pandas(desc=f'Messages in channel')
  df['sentiment'] = df['message'].progress_apply(lambda x: exc_handler(predict)(x) if pd.notna(x) else np.nan)

In [ ]:
def save_channel(df, filename):
  df.to_csv(f"{OUTPUT_FOLDER}/{filename}")

In [ ]:
# save_channel(test_df, channels[0])

In [ ]:
ch_subset = channels[20:]
print(ch_subset[0])

pushilindenis.csv


In [ ]:
for i, ch_name in enumerate(ch_subset):
  print(f"\n{i+1}/{len(ch_subset)} {ch_name}")
  df = read_channel(f"{INPUT_FOLDER}/{ch_name}")
  preprocess_df(df, ch_name)
  process_df(df)
  save_channel(df, ch_name)


1/279 pushilindenis.csv


Messages in channel: 100%|██████████| 2937/2937 [00:29<00:00, 98.15it/s]



2/279 SIL0VIKI.csv


Messages in channel: 100%|██████████| 56756/56756 [13:00<00:00, 72.67it/s]



3/279 razvedkavperedZ.csv


...

Messages in channel: 100%|██████████| 23505/23505 [05:36<00:00, 69.76it/s]



52/279 vchkogpu.csv


Messages in channel: 100%|██████████| 34043/34043 [08:50<00:00, 64.14it/s]



53/279 oper_goblin.csv


Messages in channel:  98%|█████████▊| 19951/20388 [03:08<00:08, 52.75it/s]

In [ ]:
# 32/279 Soldieroffortune777.csv